In [4]:
import base64
import datetime
import gc
import json
import pickle
import random
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from numpy import median
import matplotlib.dates as mdates

from xgboost import XGBClassifier
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

np.random.seed(1337)
%matplotlib inline
sns.set(font_scale=1.5)
rcParams['figure.figsize'] = 12, 8
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

In [5]:
def pk(true, pred, k):
    pred = pred[:k]
    if k <= len(pred) and pred[k-1] in true:
        return 1.0 * len(set(true) & set(pred)) / len(pred)
    else:
        return 0.0
def apk(true, pred, k):
    if true == '' or pred == '':
        return 0.0
    true = true.split(' ')
    pred = pred.split(' ')
    m = len(true)
    n = len(pred)
    s = 0.0
    for i in range(1, 8):
        s += pk(true, pred, i)
    return 1.0 * s / min(m, 7)

In [6]:
def rank(s):
    return ' '.join(s.sort_values(ascending=False).index[:7])

In [7]:
products = '''ind_recibo_ult1
ind_cco_fin_ult1
ind_nom_pens_ult1
ind_nomina_ult1
ind_cno_fin_ult1
ind_tjcr_fin_ult1
ind_ecue_fin_ult1
ind_dela_fin_ult1
ind_reca_fin_ult1
ind_ctma_fin_ult1
ind_valo_fin_ult1
ind_fond_fin_ult1
ind_deco_fin_ult1
ind_ctop_fin_ult1
ind_ctpp_fin_ult1
ind_plan_fin_ult1
ind_ctju_fin_ult1
ind_deme_fin_ult1
ind_cder_fin_ult1
ind_pres_fin_ult1
ind_hip_fin_ult1
ind_viv_fin_ult1
ind_aval_fin_ult1
ind_ahor_fin_ult1'''.split('\n')

In [ ]:
models = {}
for p in products:
    print p
    d = pd.read_csv('tmp/dmy_ho_train_{0}.csv'.format(p))
    del d['customer_id']
    del d['target_name']
    l = d.target.copy()
    del d['target']
    m = XGBClassifier(
                n_estimators=50,
                learning_rate=0.01,
                max_depth=9,
                gamma=0,
                min_child_weight=1,
                scale_pos_weight=5.0,
                nthread=2)
    m.fit(d, l)
    del d
    del l
    gc.collect()
    models[p] = m

In [ ]:
pickle.dump(models, open('tmp/models_xgb_ne50_spw5.pickle', 'w'))

In [8]:
models = pickle.load(open('tmp/models_xgb_ne50_spw5.pickle', 'r'))

In [9]:
t = pd.read_csv('tmp/ho_targets.csv')
t.target.fillna('', inplace=True)
t_ids = t[['customer_id']].copy()
del t['customer_id']
t.sum().sort_values(ascending=False)

target               ind_tjcr_fin_ult1ind_recibo_ult1ind_recibo_ult...
ind_recibo_ult1                                                  10163
ind_nom_pens_ult1                                                 5513
ind_nomina_ult1                                                   5488
ind_tjcr_fin_ult1                                                 4248
ind_cco_fin_ult1                                                  3854
ind_ecue_fin_ult1                                                 2709
ind_cno_fin_ult1                                                  2346
ind_ctma_fin_ult1                                                  512
ind_reca_fin_ult1                                                  279
ind_ctop_fin_ult1                                                  226
ind_valo_fin_ult1                                                  183
ind_ctpp_fin_ult1                                                  131
ind_fond_fin_ult1                                                   60
ind_de

In [10]:
d = pd.read_csv('tmp/dmy_ho_test.csv')
d = pd.merge(t_ids, d, how='left', on='customer_id')
del d['customer_id']
del d['target_name']
del d['target']

In [11]:
s = pd.DataFrame()
for p in products:
    s[p] = models[p].predict_proba(d)[:,1]
del d
s.sum().sort_values(ascending=False)

ind_recibo_ult1      352932.28125
ind_nom_pens_ult1    337908.62500
ind_nomina_ult1      336830.21875
ind_cno_fin_ult1     333024.71875
ind_tjcr_fin_ult1    332657.09375
ind_reca_fin_ult1    332515.40625
ind_ecue_fin_ult1    332304.21875
ind_valo_fin_ult1    311302.87500
ind_cco_fin_ult1     307550.31250
ind_dela_fin_ult1    302531.31250
ind_fond_fin_ult1    300502.46875
ind_ctop_fin_ult1    293278.21875
ind_plan_fin_ult1    292401.71875
ind_ctma_fin_ult1    289326.12500
ind_ctpp_fin_ult1    286296.40625
ind_pres_fin_ult1    285408.40625
ind_deco_fin_ult1    283988.28125
ind_cder_fin_ult1    282589.18750
ind_deme_fin_ult1    280603.87500
ind_hip_fin_ult1     280430.18750
ind_ctju_fin_ult1    279380.87500
ind_viv_fin_ult1     279135.75000
ind_aval_fin_ult1    279106.21875
ind_ahor_fin_ult1    279106.15625
dtype: float32

In [12]:
s.to_csv('tmp/s_xgb_ne50_spw5.csv', index=False)

In [18]:
s = pd.read_csv('tmp/s_xgb_ne50_spw5.csv')
s['target'] = s.apply(lambda r: rank(r), axis=1)
true = t.target.tolist()
pred = s.target.tolist()
x = 0
for i in range(len(true)):
    x += apk(true[i], pred[i], 7)
print x / len(true)

0.0247937729507


In [ ]:
for p in products[:3]:
    cmx = pd.DataFrame(confusion_matrix(t[p], s[p].apply(lambda x: 1 if x > 0.6 else 0)))
    cmx.index.name='actual'
    cmx.columns.name='predicted'
    print p
    print 'confusion matrix:'
    print cmx
    print '=' * 20    

In [ ]:
t[t.target != ''].target.head(20)

In [ ]:
s[t.target != ''].target.head(20)

In [ ]:
true = t.target.tolist()
pred = ['ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_ult1 ind_tjcr_fin_ult1 ind_cco_fin_ult1 ind_ecue_fin_ult1 ind_cno_fin_ult1'] * len(true)
x = 0
for i in range(len(true)):
    x += apk(true[i], pred[i], 7)
print x / len(true)